<a href="https://colab.research.google.com/github/hajri94/research-project/blob/main/Organized_Model_Building_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt 
import matplotlib.pyplot as plt
from numpy import percentile

In [ ]:
# We download the excel file
from google.colab import files
uploaded = files.upload()

Saving Test dataframe 10259.xlsx to Test dataframe 10259.xlsx


In [ ]:
# Convert the excel file into pandas Dataframe
import io 
df = pd.read_excel(io.BytesIO(uploaded['Test dataframe 10259.xlsx']))
# We round the columns (POWER , Temperature, Irradiation.flux) to one decimal place.
df['Temperature']=df['Temperature'].round(1)
df['Irradiation.flux']=df['Irradiation.flux'].round(1)
df

,READ_DATE,Week_Day,POWER,Temperature,Irradiation.flux
0,2019-01-01 00:00:00,TUE,23.20,-1.1,4.2
1,2019-01-01 01:00:00,TUE,18.40,-0.9,4.2
2,2019-01-01 02:00:00,TUE,18.90,-0.8,4.1
3,2019-01-01 03:00:00,TUE,21.40,-0.6,4.0
4,2019-01-01 04:00:00,TUE,18.83,-0.1,4.9
...,...,...,...,...,...
8755,2019-12-31 19:00:00,TUE,18.10,0.5,4.3
8756,2019-12-31 20:00:00,TUE,17.90,-0.7,3.9
8757,2019-12-31 21:00:00,TUE,22.40,-1.3,4.1
8758,2019-12-31 22:00:00,TUE,17.50,-1.7,4.0


In [ ]:
# We create multiple POWER lagged columns (we choose 12 lagged dependent variables  ) 
lagged1 = pd.concat([df['POWER'].shift(lag).rename('{}_lag{}'.format(df['POWER'].name, lag)) for lag in range(13)], axis=1)
# We drop the NaN values from the dataframe 
lagged1 = lagged1.dropna()
# we reset the indices 
lagged1 = lagged1.reset_index(drop = True)
# We create multiple Temperature lagged columns (we choose 24 lagged independent variables)
lagged2 = pd.concat([df['Temperature'].shift(lag).rename('{}_lag{}'.format(df['Temperature'].name, lag)) for lag in range(25)], axis=1)
# We drop the NaN values from the dataframe 
lagged2 = lagged2.dropna()
# we reset the indices 
lagged2 = lagged2.reset_index(drop = True)
# We create multiple Irradiation.flux lagged columns(we choose 24 lagged independent variables)
lagged3 = pd.concat([df['Irradiation.flux'].shift(lag).rename('{}_lag{}'.format(df['Irradiation.flux'].name, lag)) for lag in range(25)], axis=1)
# We drop the NaN values from the dataframe 
lagged3 = lagged3.dropna()
# we reset the indices 
lagged3 = lagged3.reset_index(drop = True)
# Maximize the size of Column in google colab 
#(we resize the maximum number of columns in google colab to fit our dataframe)
from google.colab.data_table import DataTable
DataTable.max_columns = 300
# Joining the DataFrame of POWER Lagged Columns to the Dataframe of Temperature Lagged Columns 
# We use the concat function to join 2 dataframes lagged1 (POWER) + lagged2 (Temperature) => Creation of new dataframe called "result".
result = pd.concat([lagged1,lagged2], axis=1, join='inner')
# We drop the NaN values from the dataframe 
result = result.dropna()
# we reset the indices 
result = result.reset_index(drop = True)
#Joining the DataFrame of POWER and Temperature Lagged Columns (POWER+Temperature=> result dataframe) to the Dataframe of Irradiation flux Lagged Columns 
# We use the concat function to join 2 dataframes result (POWER+Temperature) + lagged3 (Irradiation flux)  => Creation of new dataframe called "result1"
result1 = pd.concat([result,lagged3], axis=1, join='inner')
# We drop the NaN values from the dataframe 
result1 = result1.dropna()
# we reset the indices 
result1 = result1.reset_index(drop = True)
# We join the "Result1" Dataframe (Power , Temperature, Irradiation flux ) to the original dataframe (the one that we have uploaded from excel)
# we perform this line of code in order to link the two columns (READ_DATE),(Week_Day) to the datframe (result2). 
# The new dataframe need to be more organized as we see some columns that are repeated. 
result2 = pd.concat([result1,df], axis=1, join='inner')
#we remove the columns titled (POWER , Temperature, Irradiation flux)
dk=result2.drop(['POWER', 'Temperature','Irradiation.flux'], axis = 1) 
# To move the Read_Date and Week Date columns to first Position we implement some initial step: 
# First we remove the targeted column "Week_Day" from its intial position in the dataframe.  
first_column = dk.pop('Week_Day')
# Second we insert the removed column "Week_Day" to first position.
dk.insert(0, 'Week_Day', first_column)
# We perform the previous step for the "READ_DATE" column.
second_column = dk.pop('READ_DATE')
dk.insert(0, 'READ_DATE', second_column)
# We drop the NaN values from the dataframe 
dk = dk.dropna()
# we reset the indices 
dk = dk.reset_index(drop = True)
dk

,READ_DATE,Week_Day,POWER_lag0,POWER_lag1,POWER_lag2,POWER_lag3,POWER_lag4,POWER_lag5,POWER_lag6,POWER_lag7,...,Irradiation.flux_lag15,Irradiation.flux_lag16,Irradiation.flux_lag17,Irradiation.flux_lag18,Irradiation.flux_lag19,Irradiation.flux_lag20,Irradiation.flux_lag21,Irradiation.flux_lag22,Irradiation.flux_lag23,Irradiation.flux_lag24
0,2019-01-01 00:00:00,TUE,19.10,19.20,19.10,17.50,18.50,18.60,17.70,18.40,...,8.8,4.7,4.4,4.7,5.1,4.9,4.0,4.1,4.2,4.2
1,2019-01-01 01:00:00,TUE,18.73,19.10,19.20,19.10,17.50,18.50,18.60,17.70,...,22.7,8.8,4.7,4.4,4.7,5.1,4.9,4.0,4.1,4.2
2,2019-01-01 02:00:00,TUE,17.90,18.73,19.10,19.20,19.10,17.50,18.50,18.60,...,28.4,22.7,8.8,4.7,4.4,4.7,5.1,4.9,4.0,4.1
3,2019-01-01 03:00:00,TUE,18.50,17.90,18.73,19.10,19.20,19.10,17.50,18.50,...,40.0,28.4,22.7,8.8,4.7,4.4,4.7,5.1,4.9,4.0
4,2019-01-01 04:00:00,TUE,17.30,18.50,17.90,18.73,19.10,19.20,19.10,17.50,...,43.2,40.0,28.4,22.7,8.8,4.7,4.4,4.7,5.1,4.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8399,2019-12-16 23:00:00,MON,16.80,16.18,16.61,15.65,14.80,14.70,15.00,14.60,...,3.9,3.8,3.7,3.4,4.4,3.6,4.4,5.2,4.8,4.6
8400,2019-12-17 00:00:00,TUE,14.04,16.80,16.18,16.61,15.65,14.80,14.70,15.00,...,3.8,3.9,3.8,3.7,3.4,4.4,3.6,4.4,5.2,4.8
8401,2019-12-17 01:00:00,TUE,16.30,14.04,16.80,16.18,16.61,15.65,14.80,14.70,...,3.9,3.8,3.9,3.8,3.7,3.4,4.4,3.6,4.4,5.2
8402,2019-12-17 02:00:00,TUE,15.00,16.30,14.04,16.80,16.18,16.61,15.65,14.80,...,4.8,3.9,3.8,3.9,3.8,3.7,3.4,4.4,3.6,4.4


In [ ]:
# After creating the new dataframe (called dk), we link it to calendar data table  which is put in an excel file to carry out this step :  
# We download the calendar data excel file
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: ignored

In [ ]:
# Convert the excel file into Dataframe
import io 
dm = pd.read_excel(io.BytesIO(uploaded['calendar Data .xlsx']))
#In the "Bank Holidays" column we replace the NaN value with zeros.
dm['Bank Holidays'] = dm['Bank Holidays'].fillna(0)
# we remove the "READ_DATE" and "Week_Day" columns to avoid the Repetition of these column when linking it to the dataframe "Dk" 
remove_column = dm.pop('READ_DATE')
remove2_column = dm.pop('Week_Day')
# we join the the calendar data dataframe uploaded from excel file to the dataframe "Dk" 
result3 = pd.concat([dk,dm], axis=1, join='inner')
# We drop the NaN values from the dataframe 
result3 = result3.dropna()
# we reset the indices 
result3 = result3.reset_index(drop = True)
#we move the postion of  "Hour_day" column to first position .
column = result3.pop('Hour_day')
result3.insert(0, 'Hour_day', column)
# With this code we can perform the fitering of each of the "Hour_day" value.  
HourWeek=result3[result3['Hour_day']== 1]
HourWeek

,Hour_day,READ_DATE,Week_Day,POWER_lag0,POWER_lag1,POWER_lag2,POWER_lag3,POWER_lag4,POWER_lag5,POWER_lag6,...,Monday 15h,Monday 16h,Monday 17h,Monday 18h,Monday 19h,Monday 20h,Monday 21h,Monday 22h,Monday 23h,Bank Holidays
1,1,2019-01-01 01:00:00,TUE,18.73,19.10,19.20,19.10,17.50,18.50,18.6,...,0,0,0,0,0,0,0,0,0,1.0
25,1,2019-01-02 01:00:00,WED,17.70,19.70,17.60,17.80,18.91,17.90,18.1,...,0,0,0,0,0,0,0,0,0,0.0
49,1,2019-01-03 01:00:00,THU,24.00,22.77,22.95,22.46,22.30,23.30,22.3,...,0,0,0,0,0,0,0,0,0,0.0
73,1,2019-01-04 01:00:00,FRI,23.30,24.50,26.30,24.10,22.10,28.80,24.2,...,0,0,0,0,0,0,0,0,0,0.0
97,1,2019-01-05 01:00:00,SAT,30.00,30.10,27.10,23.90,23.40,22.70,25.6,...,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8305,1,2019-12-13 01:00:00,FRI,19.00,17.50,18.10,18.20,18.20,16.60,22.3,...,0,0,0,0,0,0,0,0,0,0.0
8329,1,2019-12-14 01:00:00,SAT,20.90,22.40,18.05,19.50,19.60,20.70,16.7,...,0,0,0,0,0,0,0,0,0,0.0
8353,1,2019-12-15 01:00:00,SUN,17.60,18.90,18.60,18.59,17.40,16.30,18.1,...,0,0,0,0,0,0,0,0,0,0.0
8377,1,2019-12-16 01:00:00,MON,16.80,17.90,20.40,15.80,16.20,15.65,15.9,...,0,0,0,0,0,0,0,0,0,0.0
